In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

#웹브라우저를 실행 할 때 사용할 옵션 - 사람인 것처럼 정보 입력
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/141.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )
#웹브라우저에서 url 열기
driver.get("https://play.google.com/store/apps/details?id=viva.republica.toss")

In [15]:
import time

In [16]:
driver.execute_script(f"window.scrollTo(0,1400)")
time.sleep(2)

In [17]:
# 평점 및 리뷰 옆의 -> 버튼 클릭 
button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label *= "평점 및 리뷰 자세히 알아보기"]')
button.click()

In [18]:
# 최신순으로 리뷰를 정렬하기 위해서 버튼 클릭
driver.find_element(By.ID, "sortBy_1").click()

In [19]:
# '최신'을 찾아서 클릭
driver.find_element(By.CSS_SELECTOR,'span[aria-label *= "최신"]').click()

In [20]:
#driver.execute_script(f"window.scrollTo(0,2400)")
#time.sleep(2)
# window를 스크롤하면 리뷰가 아닌 -> background 스크롤 됨

In [23]:
# 리뷰가 담긴 창을 찾아서 Javascript로 1000px씩 아래로 스크롤
driver.execute_script("document.querySelector('.fysCi.Vk3ZVd').scrollBy(0,10000)")

In [26]:
# 리뷰일
review_date = driver.find_element(By.CSS_SELECTOR, ".bp9Aid").get_attribute('innerHTML')

In [35]:
# 별점
rating = float(driver.find_element(By.CSS_SELECTOR, 'div[aria-label*="별표 5개 만점에"]').get_attribute('aria-label').split()[3].replace("개를",""))

In [41]:
# 사용자 리뷰(텍스트)
user_review = driver.find_element(By.CSS_SELECTOR,".h3YV2d").get_attribute('innerHTML')

In [40]:
# 회사 응답
company_reply = driver.find_element(By.CSS_SELECTOR,".ras4vb > div").get_attribute('innerHTML')

In [50]:
from datetime import datetime, timedelta

In [48]:
# 추출한 리뷰일을 날짜형 데이터로 변경
# datetime.strptime(yyyy-mm-dd, "%Y-%m-%d") 날짜형 데이터 타입으로 변환
review_date = review_date.replace(" ","").replace("년","-").replace("월","-").replace("일","")
review_date = datetime.strptime(review_date,"%Y-%m-%d")

In [56]:
review_date.date()

datetime.date(2025, 10, 1)

In [54]:
today = datetime.today()
one_month_ago = today - timedelta(days=30)
one_month_ago.date()

datetime.date(2025, 9, 27)

In [55]:
review_date.date() > one_month_ago.date()

True

In [58]:
#if review_date.date() < one_month_ago.date():
#    break

# 코드 합쳐서 토스 리뷰 1달치 수집하기

웹브라우저에 표시되는 요소들(버튼, 아이콘)은 클릭할 수 있게 되기까지 시간이 필요.<br>
요소가 클릭이 가능해질 때까지 기다리도록 코드를 작성해 주어야 함<br>
```python
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
```
추가가 필요

In [66]:
import time
import pandas as pd
from datetime import datetime,timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

1. 스크롤을 내린 후 마지막 자료의 날짜를 찾아서 오늘 날짜부터 1주일 전 날짜와 비교하기
2. 1주일 전 날짜가 되면 스크롤 중지(그 전까지는 반복문으로 스크롤 계속 내리기)
3. 전체 자료 수집

In [79]:
def to_date(review_date):
    # 추출한 리뷰일을 날짜형 데이터로 변경
    # datetime.strptime(yyyy-mm-dd, "%Y-%m-%d") 날짜형 데이터 타입으로 변환
    review_date = review_date.replace(" ","").replace("년","-").replace("월","-").replace("일","")
    review_date = datetime.strptime(review_date,"%Y-%m-%d")
    return review_date.date()

In [81]:
apps = dict(토스="viva.republica.toss", kb스타뱅킹="com.kbstar.kbbank", 하나원큐="com.kebhana.hanapush",
           뱅크샐러드 = "com.rainist.banksalad2", 핀다="kr.co.finda.finda")

In [85]:
def app_review_extractor(apps):
    #웹브라우저를 실행 할 때 사용할 옵션 - 사람인 것처럼 정보 입력
    options = Options()
    options.add_experimental_option("detach", True)
    options.add_argument("start-maximized")
    options.add_argument("Chrome/141.0.0.0")
    options.add_argument("lang=ko_KR")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )
    #웹브라우저에서 url 열기
    driver.get(f"https://play.google.com/store/apps/details?id={app[1]}")

    # 요소가 실행 가능해질 때까지 기다리기
    wait = WebDriverWait(driver, 10)

    # 평점 및 리뷰 옆의 -> 버튼이 활성화 되도록 스크롤을 1400px 아래로 내림
    driver.execute_script(f"window.scrollTo(0,1400)")
    time.sleep(2)

    # 평점 및 리뷰 옆의 -> 버튼 클릭 
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label *= "평점 및 리뷰 자세히 알아보기"]')))
    button.click()

    # 최신순으로 리뷰를 정렬하기 위해서 버튼 클릭
    wait.until(EC.element_to_be_clickable((By.ID, "sortBy_1"))).click()
    time.sleep(2)

    # '최신'을 찾아서 클릭
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'span[aria-label *= "최신"]'))).click()
    time.sleep(2)

    # 스크롤을 내린 후 마지막 자료의 날짜를 찾아서 오늘 날짜부터 1주일 전 날짜와 비교하기
    # 첫 페이지 마지막 리뷰의 날짜 구하기 오늘 날짜와 비교
    # 스크롤 전과 후의 리뷰 개수 비교
    today = datetime.today()
    today = today.date()
    end_date = today - timedelta(days=7)
    end_px = 1000
    len_review_list = len(driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe"))
    last_review = driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe")[-1]
    last_review_date = to_date(last_review.find_element(By.CSS_SELECTOR, ".bp9Aid").get_attribute('innerHTML'))

    while last_review_date >= end_date:
    # 리뷰가 담긴 창을 찾아서 Javascript로 1000px씩 아래로 스크롤
        driver.execute_script(f"document.querySelector('.fysCi.Vk3ZVd').scrollTo(0,{end_px})")
        time.sleep(2)
        last_review = driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe")[-1]
        last_review_date = to_date(last_review.find_element(By.CSS_SELECTOR, ".bp9Aid").get_attribute('innerHTML'))

        end_px += 10000
        print(len(driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe")))
        print(f"last_review_date: {last_review_date}, end_date: {end_date}")

    # 리뷰 목록 가져오기 div.RHo1pe
    review_list =driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe")

    result = {}
    cols = ('리뷰일','앱 이름','별점','사용자 리뷰','회사 응답')
    for review in review_list:

        # 리뷰일
        review_date = review.find_element(By.CSS_SELECTOR, ".bp9Aid").get_attribute('innerHTML')

        # 별점
        rating = float(review.find_element(By.CSS_SELECTOR, 'div[aria-label*="별표 5개 만점에"]').get_attribute('aria-label').split()[3].replace("개를",""))

        # 사용자 리뷰(텍스트)
        user_review = review.find_element(By.CSS_SELECTOR,".h3YV2d").get_attribute('innerHTML')

        # 회사 응답
        try:
            company_reply = review.find_element(By.CSS_SELECTOR,".ras4vb > div").get_attribute('innerHTML')
        except:
            company_reply = "회사 응답 없음"

        values = (review_date, apps[0], rating, user_review, company_reply)
        for key, value in zip(cols, values):
            result.setdefault(key, []).append(value)

    driver.close()        
    df = pd.DataFrame(result)
    return df

In [87]:
for app in apps.items():
    df = app_review_extractor(app)
    display(df) #print문이랑 똑같은데, 주피터에서 쓰면 깔끔하게 나옴
    df.to_csv(f"./data/{app[0]}_앱리뷰모음.csv", index=False, encoding="utf-8-sig")

20
last_review_date: 2025-10-25, end_date: 2025-10-20
40
last_review_date: 2025-10-24, end_date: 2025-10-20
60
last_review_date: 2025-10-23, end_date: 2025-10-20
80
last_review_date: 2025-10-22, end_date: 2025-10-20
100
last_review_date: 2025-10-21, end_date: 2025-10-20
120
last_review_date: 2025-10-21, end_date: 2025-10-20
140
last_review_date: 2025-10-20, end_date: 2025-10-20
160
last_review_date: 2025-10-19, end_date: 2025-10-20


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025년 10월 26일,토스,1.0,신분증 촬영 모듈 좀 바꿔주세요... 신분증 인식이 안되서 몇십분 소비합니다. . ...,"안녕하세요. 전준환님, 토스팀입니다. 만족스러운 서비스를 제공하기 위해 노력하였으나..."
1,2025년 10월 26일,토스,1.0,비번이 틀리다고 해서 못들어 갔는데 어찌해야죠,"안녕하세요. 이상준 님, 토스팀입니다. 비밀번호가 누적으로 5회 이상 오입력이 된 ..."
2,2025년 10월 26일,토스,1.0,작동이중간에멍춤,"안녕하세요. Young Cho 님, 토스팀입니다. 우선 원활한 이용에 불편을 드려 ..."
3,2025년 10월 26일,토스,1.0,진짜 리뷰 처음 써보는데 이게 맞는지 모르겠습니다 버그 다 잡았다면서 업데이트 돌리...,"안녕하세요. DH K 님, 토스팀입니다. 만족스러운 서비스를 제공하기 위해 노력하였..."
4,2025년 10월 26일,토스,3.0,교통카드 충전하려는데 왜 자꾸 카드를 인식할수 없다는지 계속 뜨고 가만히 있어도 오...,"안녕하세요. 송민건 님, 토스팀입니다. 만족스러운 서비스를 제공하기 위해 노력하였으..."
...,...,...,...,...,...
155,2025년 10월 20일,토스,1.0,아니..업데이트 왜이래여 토스증권 개불편,"안녕하세요. 킹슬라임님, 토스팀입니다. 우선 토스증권 서비스 이용에 불편을 드려 대..."
156,2025년 10월 19일,토스,3.0,업데이트후 글씨가 작아서 불편감이 있습니다... 40대라 그런지 힘드네요ㅠㅠ 글씨 ...,"안녕하세요. 순두부별똥별님, 토스팀입니다.먼저 토스를 이용해주셔서 감사합니다.\n앱..."
157,2025년 10월 19일,토스,3.0,업데이트도 안되고 실행도 갑자기 안되는데 왜 그런걸까요?,"안녕하세요. 어깨펴님, 토스팀입니다.기기에 캐시 및 데이터가 많아 앱 설치/업데이트..."
158,2025년 10월 19일,토스,3.0,업데이트 하라고 떠서 하려고 하는데 업데이트도 안되고 어플 자체가 들어가 지지 않아...,"안녕하세요. so p님, 토스팀입니다.기기에 캐시 및 데이터가 많아 앱 설치/업데이..."


20
last_review_date: 2025-10-23, end_date: 2025-10-20
40
last_review_date: 2025-10-22, end_date: 2025-10-20
60
last_review_date: 2025-10-19, end_date: 2025-10-20


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025년 10월 26일,kb스타뱅킹,5.0,최고,"안녕하세요, 고객님. KB스타뱅킹을 이용해주시고, 바쁘신 시간 중에도 소중한 칭찬글..."
1,2025년 10월 26일,kb스타뱅킹,1.0,국민인증서로 아무리 인증해도 줄 세개짜리 메뉴에 들아가면 로그아웃되어 있어요 초기화...,"안녕하세요, 고객님. KB스타뱅킹 이용에 불편을 드려 죄송합니다. 남겨주신 리뷰만으..."
2,2025년 10월 26일,kb스타뱅킹,2.0,김지정입니다 제가것요 사용중 물건손대지말세요 제가관리합니다 ⚠️ ⛔️ 📵 금지요 저...,"안녕하세요 고객님, KB스타뱅킹 이용에 불편을 드려 죄송합니다. 고객님께서 남겨주신..."
3,2025년 10월 25일,kb스타뱅킹,5.0,편함,고객님 안녕하세요! KB스타뱅킹을 이용해 주시고 바쁘신 와중에 칭찬글까지 남겨주시어...
4,2025년 10월 25일,kb스타뱅킹,1.0,자꾸 실행되지도 않고 팅김,"고객님 안녕하세요, KB스타뱅킹 이용에 불편을 드려 대단히 죄송합니다. 고객님께서 ..."
5,2025년 10월 25일,kb스타뱅킹,1.0,업데이트를 하면 쓰레기 같이 변하는 앱,"고객님, 안녕하세요. 저희 KB스타뱅킹을 이용하셨으나 만족스러운 금융서비스 경험을 ..."
6,2025년 10월 25일,kb스타뱅킹,5.0,좋아요,"안녕하세요, 고객님. KB스타뱅킹을 꾸준히 이용해주시고 소중한 시간을 내어 칭찬 글..."
7,2025년 10월 25일,kb스타뱅킹,5.0,아주좋아요,고객님 안녕하세요! KB스타뱅킹을 이용해 주시고 바쁘신 와중에 칭찬글까지 남겨주시어...
8,2025년 10월 25일,kb스타뱅킹,5.0,굿~~~,"고객님, 안녕하세요! KB스타뱅킹을 이용해 주셔서 진심으로 감사합니다. 앞으로도 고..."
9,2025년 10월 25일,kb스타뱅킹,1.0,아. 답이없네 진짜 이건 아니지 이건 아니야 인증을 수십번을해도 화면꺼지고 뭐 이런...,"고객님, 안녕하세요. KB스타뱅킹 이용에 불편을 드려 죄송합니다. KB스타뱅킹이 백..."


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025년 10월 24일,하나원큐,1.0,예수금 빼려고 보니까 그냥은 또 안빠지고 계좌 밀어야 된데서 계좌 해지 찾아보니 밀...,회사 응답 없음
1,2025년 10월 24일,하나원큐,1.0,앱이 왜 안열리져 다른통장에서 이체를해서 볼려고해도볼수가없네여 출근 교통비 써야되는...,회사 응답 없음
2,2025년 10월 24일,하나원큐,1.0,갤럭시 s24 울트라 순정 os 씁니다. 얼마전부터 접속하려고 할 때마다 개조된 o...,회사 응답 없음
3,2025년 10월 23일,하나원큐,1.0,로그인 인증이 안되어서 삭제후 재설치 했더니 아예 안되네요,회사 응답 없음
4,2025년 10월 22일,하나원큐,4.0,"입출금알림 요즘 너무너무너무늦게와요 실시간 알림아니면 무슨 의미죠..,",회사 응답 없음
5,2025년 10월 22일,하나원큐,5.0,"삼성 a 시리즈 사용중인데, 루팅 하지도 않았는데 루팅폰이라고 쟈꾸 뜹니다.",안녕하세요. 하나은행입니다. 이용 중 불편을 드려 죄송합니다. 현재 해당 현상은 최...
6,2025년 10월 22일,하나원큐,1.0,one ui 8.0 업데이트 이후 정상적인 OS 에서만 사용가능하다는 알림창 뜨면서...,회사 응답 없음
7,2025년 10월 22일,하나원큐,1.0,OS개조한적 없는데 루팅폰이라고 자꾸 떠서 삭제했다 재설치해야했음. 당연히 최신버전...,안녕하세요. 하나은행입니다. 이용 중 불편을 드려 죄송합니다. 현재 해당 현상은 최...
8,2025년 10월 21일,하나원큐,1.0,인증을 도대체 몇번이나 시키는거며 매끄럽게 진행되는게 없네요 최악입니다,안녕하세요. 하나은행입니다. 우선 이용에 불편을 드려 죄송합니다. 앱리뷰로는 정확한...
9,2025년 10월 21일,하나원큐,1.0,왜 안될까요? 첫 화면에서 계속 돌기만하네요~안되서 또 지워서 낄아도안되네요~다른 ...,안녕하세요. 하나은행입니다. 서비스 이용에 불편을 드려 죄송합니다. 계속적으로 동일...


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025년 10월 26일,뱅크샐러드,5.0,5년 넘게 제 가계부로 잘 쓰고 있습니다. 항상 잘 쓰고 있습니다. 추가 되었으면 ...,"안녕하세요, 뱅크샐러드 고객감동팀입니다. 아직 문제가 해결되지 않아 많이 답답하셨을..."
1,2025년 10월 25일,뱅크샐러드,1.0,"업데이트 후 메모•태그 입력하는데 오래걸리고, 자주 오류나서 불편합니다. 메모 기능...",회사 응답 없음
2,2025년 10월 25일,뱅크샐러드,5.0,빗썸도 가상자산 연결할수있게해주세요,회사 응답 없음
3,2025년 10월 25일,뱅크샐러드,1.0,업데이트후 계속 꺼짐 실행시키면 계속 꺼집니다 인터넷이 원활하지 않다는 메세지도 가...,회사 응답 없음
4,2025년 10월 23일,뱅크샐러드,1.0,업데이트 후 계속 버벅이고 오류나요.,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
5,2025년 10월 23일,뱅크샐러드,4.0,국민연금 납부액과 예상수령액도 연동되게해주세요,안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드에 국민연금을 연결해 납부액과 예...
6,2025년 10월 22일,뱅크샐러드,4.0,"너무너무 잘쓰고 있었는데 코나아이, 투자 부분에 대한 관리가 굉장히 미흡합니다 코나...","안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 애정하시고, 소중한 의견 전달..."
7,2025년 10월 20일,뱅크샐러드,4.0,취소 내역이 섞여 있으면 가계부가 뭔가 미묘하게 맞지 않는 것 같습니다 음...,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
8,2025년 10월 18일,뱅크샐러드,4.0,최근 로그인 한 지 얼마 지나지 않았는데도 장시간 이용이 없어서 로그아웃 되었다고 ...,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
9,2025년 10월 18일,뱅크샐러드,3.0,업데이트 후 날짜 변경시 캘린더뷰가 아니라서 너무 답답합니다... 캘린더앱을 보러가...,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025년 10월 23일,핀다,5.0,생각지도 못한자금이 생겼네요 다른곳에선 이한도 이이자율 안나왔는데 핀다 짱이네요,"안녕하세요, 안은경님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
1,2025년 10월 23일,핀다,5.0,최고예요.은행.보험도 안되는걸 판다는 해내요!!,"안녕하세요, 더키님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼저..."
2,2025년 10월 21일,핀다,1.0,저는대출이낮다고거절돼요,"안녕하세요, 장윤정님. 절박한 상황에서 대출이 거절되어 많이 실망하셨을 것 같아 죄..."
3,2025년 10월 19일,핀다,1.0,신분증 찰영 프로그램 정말 쓰레기네요 몇백번을 찍어도 인식을 못하는,"안녕하세요 이승원 고객님, 핀다 고객경험팀입니다. 대출 신청 중 신분증 촬영이 되지..."
4,2025년 10월 16일,핀다,5.0,핀다에서 대출500백받았습니다 감사합니다,"안녕하세요, 전현숙님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
5,2025년 10월 15일,핀다,5.0,이자율도 싸고. 빠르고 편리하게. 받았어요 핀다에. 감사해요,"안녕하세요, 최종도님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
6,2025년 10월 14일,핀다,1.0,신용정보 검색을 자꾸 시키는 어이없네 대출 막으려고 하는짓인,"안녕하세요 sunny k 고객님, 신용조회 오해에 대해 안내드립니다. 핀다는 신용조..."
7,2025년 10월 14일,핀다,4.0,힘들때ㅡ감사힙니다,"안녕하세요, 김선오님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
8,2025년 10월 14일,핀다,3.0,앱을 다운 받고 실행하는데 앱을 종료하였습니다 라고 뜨는데 오류인건가여 대출 알아 ...,"심지혜 고객님 안녕하세요, 핀다 고객경험팀입니다. 핀다 앱을 실행하는 과정에서 앱이..."
9,2025년 10월 13일,핀다,5.0,아픈사람이 있어서 대출을 받기로 했는데 추석 연휴 로 좀 시간이 흘러서 맘이 몹시 ...,"안녕하세요, 앙주인숙님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 ..."


## DB로 저장하기

from dbio2 import to_db, db_connect

In [ ]:
# --- 4. 메인(Main) 함수: DB 연결 및 작업 실행 ---
def main():
    
    DB_NAME = "Bank_reviews" 
    
    apps = {
        "토스": "viva.republica.toss",
        "KB스타뱅킹": "com.kbstar.kbbank",
        "하나원큐": "com.kebhana.hanapush",
        "뱅크샐러드" : "com.rainist.banksalad2", 
        "핀다": "kr.co.finda.finda"
    }

    for app_tuple in apps.items():
        app_name = app_tuple[0]
        app_id = app_tuple[1]
        
        TABLE_NAME = app_id.replace(".", "_") 
        
        df = app_review_extractor(app_tuple) 
        
        if df.empty:
            continue

        try:
            to_db(DB_NAME, TABLE_NAME, df)
        except Exception:
            continue # DB 저장 실패 시 조용히 다음 앱으로 넘어감

# --- 5. 스크립트 실행 시작점 ---
if __name__ == "__main__":
    main()